In [25]:
import import_ipynb
import register as reg
import numpy as np
import math
import register
import distutils

In [57]:
class Gate(object):
    """
    Gate class, which stores a matrix that might operate on a qubit system (via its basis state amplitudes, classical analog)

        Attributes
        ----------
        matrix: double array
            given a matrix of any size, this will create a new object that identifies as a Gate.

    """
    def __init__(self, gate):
        self.matrix = np.array(gate,complex)
    def __repr__(self):
        return self.matrix.__repr__()

In [62]:
# Series of gates without being in Gate class for easy use, not relevant to overall program.
idg = Gate([[1, 0], [0, 1]]) # Identity gate
s2 = 2**(-.5) 
hg = Gate([[s2, s2], [s2, -s2]]) # Hadamard gate
px = Gate([[0, 1], [1, 0]])
pz = Gate([[1,0], [0,-1]])

In [76]:
def create(qubits, gates, customkron=False):
    """
    Function that creates size appropriate gates (scaled in tensor space)
       
    This function enables several gates to be applied as one matrix, by using Kroneker's product
    "qubits" times. This is crucial in the classical setting, because we will need to find the
    total product of all the gates before applying it to an initial registery.

        Parameters:
            qubits: int
                size of the system in qubit count
            gates: double array
                which gates will be applied (matrices in this classical setting)
        Returns:
            combination: Gate class
                combined gate
    
    """
    if customkron==False:
        combination = [[1]]
        for i in range(0,qubits):
            combination = np.kron(gates[i].matrix, combination)
    else:
        combination = [[1]]
        for i in range(0,qubits):
            combination = kron(gates[i].matrix, combination)
    return Gate(combination)

In [90]:
def globalapp(qubits, mat):
    """
    Scales a gate to the largest tensor space (apply it to all qubits)
    
        Parameters:
            qubits: int
                # of qubits
            mat: double array
                lists of gates to be applied (matrices in this classical setting)
        Returns:
            combination: Gate class
                globally applicable gate
            
    """
    
    count = [mat for i in range(qubits)]
    return create(qubits,count)

In [89]:
def diffusion(qubits,customouter=False):
    """
    This is Grover's diffusion gate. It starts and ends with the Hadamard, 
    and in the middle is this reflector around |0> which is  2|s><s|-I. 
    
        Parameters:
            qubits: # of qubits
            customeouter: Boolean
                if true, use custom outer product
        Returns:
            Grover's Diffusion Gate
            
    """
    # create globally applicable hadamard and identity gates. The former will be
    # on either side of reflector, and the latter be crucial to the reflection.
    hgate = globalapp(qubits,hg)
    igate = globalapp(qubits,idg)
    
    # Create a register in zeroth state to reflect off of. Create 2|s><s|-I matrix.
    zedstate = reg.given(0,1<<qubits)
    if customouter==True:
        czero = Gate(2*outer(zedstate.var,zedstate.var)-igate.matrix)
    else:
        czero = Gate(2*np.outer(zedstate.var,zedstate.var)-igate.matrix)
    
    # Return product H -> Reflection -> H as one Diffusion matrix
    return hgate.matrix@czero.matrix@hgate.matrix

In [52]:
def idgate():
    """Returns Identity Gate"""
    return Gate([[1, 0], [0, 1]])
def hgate():
    """Returns Hadamard Gate"""
    invsqr2 = 2**(-.5)
    return Gate([[invsqr2, invsqr2], [invsqr2, -invsqr2]])

In [85]:
def outer(a,b):
    """
    Function that computer outer product
        Parameters:
            a: double array
            b: double array
        
        Returns:
            outer: double array
                outer product
    """
    outer = np.zeros((len(a),len(b)),dtype=complex)
    for i in range(len(a)):
        for j in range(len(b)):
            outer[i,j]=a[i]*b[j]
    return outer            

In [78]:
def kron(matrix_a, matrix_b):
    """
    Function that computes Kroneker's product
    
        Parameters:
            matrix_a: double array
            matrix_b: double array
        
        Returns:
            Kronecker product of two matrices
    """
    a_multiplier = np.ones((np.shape(matrix_a)[0] * np.shape(matrix_b)[0], np.shape(matrix_a)[1] * np.shape(matrix_b)[1]), dtype = complex)
    b_multiplier = np.tile(matrix_b, np.shape(matrix_a))
    for m in range(len(a_multiplier)):
        for n in range(len(a_multiplier[0])):
            a_multiplier[m][n] = matrix_a[int(m/len(matrix_b))][int(n/len(matrix_b[0]))]
    return a_multiplier*b_multiplier

In [53]:
def oracle(qubits, key):
    """Returns Oracle Gate w/Identity Method"""
    eye = np.eye(qubits,dtype=complex)
    eye[key,key]=-1.0
    return eye

In [64]:
def ft(qubits):
    """
    Returns Quantum Fourier Transform Gate
        Parameters:
            qubits: int
                # of qubits in system
        Returns:
            QFT gate: Gate class
            
    This will first create out unit omega (w)^((2)(pi)(i)/n)
    where n is basis size. It will then go through each point
    on an nxn lattice and multiply by the row/col to replicate
    a QFT. Proper normalization factor is included at the end.
    """
    n = 1<<qubits  
    w = np.exp(2.0 * np.pi * 1j / n) 
    fgate=np.zeros((n,n),complex)
    for i in range(n):
        for j in range(n):
            fgate[i,j]=(w ** (i*j))
    fgate = fgate / np.sqrt(n) 
    return Gate(fgate)